In [1]:
# BLOK 1 
# IMPORTOVANIE KNIZNIC

import pandas as pd
import numpy as np
import json
from random import randint
import matplotlib.pyplot as plt

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Convolution1D, MaxPooling1D, Flatten, Dense, Dropout, LSTM, GRU, Concatenate
import tensorflow.keras.utils 
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from ast import literal_eval
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, classification_report

from tensorflow.keras.models import Sequential

C:\Users\nemci\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\nemci\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\nemci\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\nemci\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

In [ ]:
# ----------------------- SYNTHETIC LIGHT CURVES ----------------------- #
# ----------------------- SYNTHETIC LIGHT CURVES ----------------------- #
# ----------------------- SYNTHETIC LIGHT CURVES ----------------------- #

In [2]:
# BLOK 2 
# Nacitanie datasetu syntetickych kriviek

data_syn = pd.read_csv(r"C:\Users\nemci\OneDrive\Počítač\Bakalárska práca\Data\df_all.csv_v2")

In [3]:
# BLOK 3
# Vyber podmnoziny oddelenych kriviek z datasetu syntetickych kriviek

data_det = data_syn[data_syn["morphology"] == "detached"]

print("Detached light curves: ", data_det.shape[0])

Detached light curves:  899218


In [4]:
# BLOK 4
# Diskretizacia 

newDataDet = []
for row in data_det['generic_bessell_v']:
    row = json.loads(row)
    minValueOfRow, maxValueOfRow = min(row), max(row)
    newRow = []
    for valueIndex in range(len(row)):
        row[valueIndex] = (row[valueIndex] - minValueOfRow) / (maxValueOfRow - minValueOfRow)
        newRow.append([row[valueIndex]])
    newDataDet.append(newRow)
newDataDet = np.array(newDataDet)

In [5]:
# BLOK 5
# Vytvorenie cieloveho atributu kriviek. Hodnota 0 pre neskvrnite, hodnota 1 pre skvrnite krivky

target_det = []
det_S, det_No = 0, 0
for row in data_det["spotty"]:
    if row == 1:
        target_det.append(1)
        det_S += 1
    elif row == 0:
        target_det.append(0)
        det_No += 1
target_det = np.array(target_det)
print("Detached spotty: " + str(det_S) + "\nDetached no spotty: " + str(det_No))

Detached spotty: 453662
Detached no spotty: 445556


In [6]:
# BLOK 6
# Rozdelenie upravenych kriviek na trenovaciu a testovaciu mnozinu v pomere 80:20

print("Number of detached light curves: " + str(len(newDataDet)) + "\nTarget attribute: " + str(len(target_det)))
X_trainDet, X_testDet, y_trainDet, y_testDet = train_test_split(newDataDet, target_det, test_size = 0.2)
y_trainDet = to_categorical(y_trainDet, 2)
y_testDet = to_categorical(y_testDet, 2)
print("Length X_trainDet: ", str(len(X_trainDet)))
print("Length y_trainDet: ", str(len(y_trainDet)))
print("Length X_testDet: ", str(len(X_testDet)))
print("Length y_testDet: ", str(len(y_testDet)))

Number of detached light curves: 899218
Target attribute: 899218
Length X_trainDet:  719374
Length y_trainDet:  719374
Length X_testDet:  179844
Length y_testDet:  179844


In [ ]:
# BLOK 7
# Vytvorenie architektúry klasifikačneho modelu a jeho trenovanie

model = Sequential()
model.add(Convolution1D(128, 2, activation = 'relu', input_shape = (100, 1))) 
model.add(MaxPooling1D(pool_size=2))
model.add(Convolution1D(64, 2, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Convolution1D(128, 2, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

savedModel = "new_modelDet.hdf5"
checkpoint = ModelCheckpoint(savedModel, monitor = "val_loss", verbose = 2, save_best_only = True, mode = 'min')
model.fit(X_trainDet, y_trainDet, epochs = 7, batch_size = 32, validation_split=0.2, callbacks = [checkpoint], verbose = 2)

In [7]:
# BLOK 8
# Nacitanie modelu. Predikovanie syntetických oddelených kriviek a vyhodnotenie modelu na testovacej mnozine syntetických 
# oddelených kriviek pomocou confusion matrix a vypocet presnosti, navratnosti a f1 skore

model = load_model('new_modelDet.hdf5')
y_pred1DCNN = model.predict(X_testDet)
y_pred2 = []
for i in y_pred1DCNN:
    maximum = np.argmax(i)
    y_pred2 = np.append(y_pred2, maximum)

cm = confusion_matrix(y_testDet.argmax(axis=1), y_pred2)
print("Confusion Matrix 1D CNN (detached): \n" + str(cm))

prfs = classification_report(y_testDet.argmax(axis=1), y_pred2)
print(str(prfs))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Confusion Matrix 1D CNN (detached): 
[[88815    66]
 [10708 80255]]
              precision    recall  f1-score   support

           0       0.89      1.00      0.94     88881
           1       1.00      0.88      0.94     90963

    accuracy                           0.94    179844
   macro avg       0.95      0.94      0.94    179844
weighted avg       0.95      0.94      0.94    179844

